In [1]:
import pandas as pd
import numpy as np

In [2]:
m2 = pd.read_excel('首次M2/首次M2明细表.xlsx')
zmd = pd.read_excel('首次M2/主门店汇总.xlsx')
m2 = m2[['贷款编号','贷款金额','产品名称','SA工号','SA姓名']]
m2['贷款编号'] = m2['贷款编号'].astype('O')
m2['SA工号'] = m2['SA工号'].astype('O')
zmd['贷款编号'] = zmd['贷款编号'].astype('O')

In [5]:
m2_over=pd.pivot_table(m2,index=['SA工号'],values=['贷款编号'],aggfunc=[len])
m2_over_rate = pd.read_excel("首次m2/m2_over_rate.xlsx")
m2_over = pd.merge(m2_over_rate,m2_over,on="SA工号",how="left",suffixes=('','_y'))
m2_over['首次M2逾期率'] = 0.0

d:\ProgramData\Miniconda3\envs\lihuaqi\lib\site-packages\pandas\core\reshape\merge.py:543: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [8]:
#首次M2逾期率计算
for i in range(len(m2_over)):
    m2_over.loc[i,'首次M2逾期率'] = (m2_over.iloc[i,3] /m2_over.iloc[i,2])*100

m2_over['SA工号'] = m2_over['SA工号'].astype('O')
m2_over = m2_over[['SA工号','首次M2逾期率']]

In [9]:
m2 = pd.merge(m2, zmd, on="贷款编号", how="left")
m2["扣罚"] = 0
for i in range(len(m2)):
    if m2.loc[i, '产品名称'] == '一般产品' or m2.loc[i, '产品名称'] == '优惠产品':
        if m2.loc[i, '贷款金额'] >= 1500:
            m2.loc[i, '扣罚'] = 180
        else:
            m2.loc[i, '扣罚'] = 90
    elif m2.loc[i, '产品名称'] == '003产品':
        m2.loc[i, '扣罚'] = 60
    elif m2.loc[i,'产品名称'] =="U客购":
        m2.loc[i, '扣罚'] = 300
    else:
        if m2.loc[i, '贷款金额'] >= 2300:
            m2.loc[i, '扣罚'] = 120
        elif 1800 <= m2.loc[i, '贷款金额'] < 2300:
            m2.loc[i, '扣罚'] = 80
        else:
            m2.loc[i, '扣罚'] = 40

m2["最终扣罚"] = 0
for i in range(len(m2)):
    if np.isnan(m2.loc[i, '每日提成金额']):
        m2.loc[i, '最终扣罚'] = m2.loc[i, '扣罚']
    else:
        m2.loc[i, '最终扣罚'] = m2.loc[i, '每日提成金额']

m2['逾期等级'] = '首次M2'


In [11]:
m2_SA = m2.drop_duplicates(["SA工号"])
m2_SA = m2_SA[['SA工号']]
overdue_rate_m3 = pd.read_excel("首次M2/Overdue_rate_M3.xlsx")
overdue_rate_m3['SA工号'] = overdue_rate_m3['SA工号'].astype('O')
m2_SA = pd.merge(m2_SA,m2_over,on="SA工号",how="left",suffixes=('','_y'))
m2_SA = pd.merge(m2_SA,overdue_rate_m3,on="SA工号",how="left",suffixes=('','_y'))

people_list = pd.read_excel("首次M2/people_listing.xlsx")
people_list['SA工号'] = people_list['SA工号'].astype('O')
m2_SA = pd.merge(m2_SA,people_list, on="SA工号",how="left",suffixes=('','_y'))


In [13]:
m2_SA

,SA工号,首次M2逾期率,SA姓名,M3+逾期率(%),在职天数
0,12230165,10.416667,林旺,9.16,584
1,3120142,5.405405,查里哈尔·木拉克,3.25,317
2,25100213,6.779661,杨锦锦,2.33,144
3,10030019,20.000000,靳成全,2.90,487
4,23250224,18.750000,吴彦桃,6.25,740
5,17260036,22.222222,祁增斌,7.81,235
6,11190006,3.225806,开赛尔·苏来曼,2.41,359
7,4120066,7.407407,迪丽木热提·赛皮,1.39,223
8,6240058,5.263158,奉先星,24.71,275
9,19040036,9.803922,孙冬冬,3.23,304


In [109]:
for i in range(len(m2_SA)):
    if np.isnan(m2_SA.loc[i,"首次M2逾期率"]):
        m2_SA.loc[i,"首次M2逾期率"] = 0
    else:
        pass
#为na的赋值为0
for i in range(len(m2_SA)):
    if np.isnan(m2_SA.loc[i,"M3+逾期率(%)"]):
        m2_SA.loc[i,"M3+逾期率(%)"] = 0
    else:
        pass

In [99]:
m2_SA["是否免除扣罚"] = 0
for i in range(len(m2_SA)):
    if m2_SA.loc[i,"在职天数"] >= 90 :
        if m2_SA.loc[i,"首次M2逾期率"] < 5.0:
            if m2_SA.loc[i,"M3+逾期率(%)"] < 7.0:
                m2_SA.loc[i,"是否免除扣罚"] = 1
            else:
                m2_SA.loc[i,"是否免除扣罚"] = 0
        else:
             m2_SA.loc[i,"是否免除扣罚"] = 0
    else:
        if m2_SA.loc[i,"首次M2逾期率"] < 4.0:
            m2_SA.loc[i,'是否免除扣罚'] = 1
        else:
            m2_SA.loc[i,"是否免除扣罚"] = 0


In [100]:
m2_SA = m2_SA[["SA工号","是否免除扣罚"]]
m2 = pd.merge(m2,m2_SA,on="SA工号",how="left")

,贷款编号,贷款金额,产品名称,SA工号,SA姓名,每日提成金额,扣罚,最终扣罚,逾期等级,是否免除扣罚
0,460102018030705035,3298,优惠产品,12230165,林旺,NaN,180,180,首次M2,0
1,654302018030704317,1100,优惠产品,3120142,查里哈尔·木拉克,NaN,90,90,首次M2,0
2,653102018030703295,4800,一般产品,25100213,杨锦锦,NaN,180,180,首次M2,0
3,630002018030706678,2998,一般产品,10030019,靳成全,NaN,180,180,首次M2,0
4,630102018030701855,2798,优惠产品,23250224,吴彦桃,NaN,180,180,首次M2,0
5,630102018030702975,3000,优惠产品,17260036,祁增斌,NaN,180,180,首次M2,0
6,652102018030706181,1798,优惠产品,11190006,开赛尔·苏来曼,NaN,180,180,首次M2,1
7,652102018030706419,1880,一般产品,4120066,迪丽木热提·赛皮,NaN,180,180,首次M2,0
8,530702018030704940,2498,优惠产品,6240058,奉先星,NaN,180,180,首次M2,0
9,652902018030705866,2000,一般产品,19040036,孙冬冬,NaN,180,180,首次M2,0


In [101]:
m2['SA最终扣罚'] = 0
for i in range(len(m2)):
    if m2.loc[i,'是否免除扣罚'] == 1.0:
        m2.loc[i,'SA最终扣罚'] = 0
    elif m2.loc[i,'是否免除扣罚'] == 0.0:
        m2.loc[i,'SA最终扣罚'] = m2.loc[i,'最终扣罚']
    else:
        print("免除扣罚出现问题，请核实！")

#m2.to_excel("SA首次M2单笔扣罚.xlsx")
m2.to_csv("SA首次M2单笔扣罚.csv")